

# Learning Bayesian Networks


Previous notebooks showed how Bayesian networks economically encode a probability distribution over a set of variables, and how they can be used e.g. to predict variable states, or to generate new samples from the joint distribution. This section will be about obtaining a Bayesian network, given a set of sample data. Learning a Bayesian network can be split into two problems:

 **Parameter learning:** Given a set of data samples and a DAG that captures the dependencies between the variables, estimate the (conditional) probability distributions of the individual variables.
 
 **Structure learning:** Given a set of data samples, estimate a DAG that captures the dependencies between the variables.
 
This notebook aims to illustrate how parameter learning and structure learning can be done with pgmpy.
Currently, the library supports:
 - Parameter learning for *discrete* nodes:
   - Maximum Likelihood Estimation
   - Bayesian Estimation
 - Structure learning for *discrete*, *fully observed* networks:
   - Score-based structure estimation (BIC/BDeu/K2 score; exhaustive search, hill climb/tabu search)
   - Constraint-based structure estimation (PC)
   - Hybrid structure estimation (MMHC)


## Parameter Learning

Suppose we have the following data:

### Back Pain Data Analysis

Jay Urbain, PhD
Meredith Adams, MD

10/25/2016, 11/5/2016, 11/21/2016, 11/23/2016

Integration of statistical analysis and machine learning methods to identify factors associated with opioid prescribing in NIH research standards for Low Back Pain survey data: a pilot analysis.

<a href="cLBP_RTF_MinimalDataset.pdf">Standford Back Pain Survey</a>.

<a href="data/stanford low back pain survey data.csv">Data set</a> (csv).

#### Introduction:

The recent development of research standards for low back pain (NIH LBP taskforce reference) creates an opportunity for prospective data standardization. Ultimately, the goal of this standardized data collection is to better understand patterns for treatment response and build predictive care models. While the impact of aggregate data will depend on large-scale integration, the focus of this study is to better understand the relationship and predictive ability of the survey variables, specifically examining predictors of opioid use. 
Clinical research is evolving to reflect the need for efficient clinical trial design and data collection, which is reflected by the move toward improved data standardization. A key component of this is adaptive statistical designs and analysis methods. The expert consensus panel developed the NIH task force for research standards questionnaire (LBPTF) to overcome common research barriers while addressing the underlying key clinical questions for low back pain. Specifically, it shifted the focus from anatomic or pathophysiological classification to that of pain interference, functional status, and pain intensity. This focused questionnaire measures these domains using several short forms from PROMIS (Patient-Reported Outcome Measurement Information System).

The novel organizational framework of the LBPTF questionnaire incorporates key clinical self-report measures as well as information about co-morbid conditions, demographic information, and treatment history. Understanding the co-occurence patterns of these data may provide insight into more focused data collection as well as build toward predictive modeling. The inherent limitations of self-reported data are mitigated by the extensive development of the minimum data set variables to incorporate key perceived domains of influence.

Building from this perspective, the objective of this pilot survey was to deconstruct and analyze the inter-relationship of these variables in a way that will provide more meaningful analysis of these data moving forward. Statistical analysis and interpretation can be misleading due to inherent data assumptions, but with the data points selected by expert consensus, this minimum dataset represents the starting point for analyzing these relationships. Recognizing the limitations of a survey snapshot, we planned iterative analyses of a pilot survey obtained during the LBPTF with several statistical and machine learning methods to validate our approach.

#### Results Summary

We performed a preliminary statistical analysis of the back pain dateset.

For the 200-sample dataset, Opioid use negatively affects the population distribution for physical function, physical interference, and pain intensity. All of these results are statistically significant (*p* << 0.05*) using Pearson's $\chi^2$ (Chi-squared) goodness of fit statistic.

We evaluate Logistic Regression (LR), Support Vector Machines (SVM), Decision Trees (DT), and recursive feature elimination for identifying the most predictive features in the data set.

Using all features and 10-fold cross-validation, our predictive accuracies are *0.69, 0.63, and 0.58* for LR, SVM, and DT respectively. 

Using the top 10 features identified using recursive feature elimination for each classifier and 10-fold cross-validation, our predictive accuracies improvei modestly to *0.71, 0.66, and 0.66* for LR, SVM, and DT respectively.

#### Legend

Match columns in the data set to questions in the survey.


DUR - 1. How long has low-back pain been an ongoing problem for you?  
FREQ - 2. How often has low-back pain been an ongoing problem for you over the past 6 months?    
NRS - 3. In the past 7 days, how would you rate your low-back pain on average?  
RAD - 4. Has back pain spread down your leg(s) during the past 2 weeks? (radiculopathy)  
PIDAY - 9. How much did pain interfere with your day-to-day activities?  
PIWORK - 10. How much did pain interfere with work around the home?  
PPISOC - 11. How much did pain interfere with your ability to participate in social activities?  
PICHOR - 12. How much did pain interfere with your household chores?  
LBS - 6. Have you ever had a low-back operation?   
LBST - 7. If yes, when was your last back operation?  
FUS - 8. Did any of your back operations involve a spinal fusion?    
OPI - 13. Opioid painkillers, have you used for your back pain?  
INJ - 13. Injections such as epidural steroid injections, facet injections, have you used for your back pain?  
EXE - 13. Exercise therapy, have you used for your back pain?  
PSY - 13. Psychological counseling, have you used for your back pain?  
UNEMP - 14. I have been off work or unemployed for 1 month or more due to low-back pain.   
DIS - 15. I receive or have applied for disability or workers’ compensation benefits because I am unable to work due to low-back pain.  
ABD - 5. Stomach pain  
JOI - 5. Pain in your arms, legs, or joints other than your spine or back  
HEA - 5. Headaches # Widespread pain or pain in most of your body??  
FIB - 15. I receive or have applied for disability benefits because I am unable to work dueto low-back pain.  
CHOR - 16. Are you able to do chores such as vacuuming or yard work?  
STAIR - 17. Are you able to go up and down stairs at a normal pace?  
W15 - 18. Are you able to go for a walk of at least 15 minutes?  
ERANDS 19. Are you able to run errands and shop?  
WORTH -, 20. In the past 7 days, I felt worthless.  
HELPL,  21. In the past 7 days, I felt helpless.  
DEPRES -  22. In the past 7 days, I felt depressed.  
HOPEL -  23.  In the past 7 days, I felt hopeless.  
SLEEPQ - 24.  In the past 7 days, my sleep quality was (choices)  
SREFR - 25.  In the past 7 days, my sleep was refreshing.  
SPROB - 26.  In the past 7 days, I had a problem with my sleep.  
SONSET - 27. I had difficulty falling asleep  
CAT.SAFE - 28.  It’s not really safe for a person with my back problem to be physically active.  
CAT.NEVER, 29.  I feel that my back pain is terrible and it’s never going to get any better.  
LIT,  30.  Are you involved in a lawsuit or legal claim related to your back problem?  
AS -  
ETOH  - 31. Have you drunk or used drugs more than you meant to?
SAHELP – 32. Have you felt you wanted or needed to cut down on your drinking or drug use?   
AGE - 33. Age: years (0–120)   
SEX - 34. Gender (Male/Female/Unknown/Unspecified)
HIS – 35. Hispanic or Latino/Not H or L/Unknown/Unreported)  
NAT - Native American
ASA - Asian
BL - Black or African American
PAC - Native Hawaiian or Pacific Islander
W -  White
UNK - Unknown
NA. – Not reported  
EMP - 37. Employment Status  
EDU - 38. Education Level: (select the highest level attained)   
SMOK - 39. How would you describe your cigarette smoking?   
HT - 40. Height  
WT - 40. Weight  
RACE - 36.  
PI - ??  
FUN - ??  
DEP - 22. ??  
SLEEP - 24. ??  


#### Load the data

In [34]:
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline 

# Read dataset into a Pandas dataframe
df = pd.read_csv("data/stanford low back pain survey data.csv")
# Dimensions
df.shape

(200, 60)

#### Reveiw and clean the  data

In [35]:
import warnings
warnings.filterwarnings('ignore')

# show column distributions
print(df.describe())

               ID         DUR       FREQ         NRS         RAD       PIDAY  \
count  200.000000  200.000000  200.00000  199.000000  199.000000  200.000000   
mean   115.520000    3.530000    2.45000    5.492462    0.422111    3.330000   
std     68.958436    0.686627    0.72118    2.117434    0.495142    1.061477   
min      5.000000    1.000000    1.00000    1.000000    0.000000    1.000000   
25%     57.750000    3.000000    2.00000    4.000000    0.000000    3.000000   
50%    107.500000    4.000000    3.00000    6.000000    0.000000    3.000000   
75%    172.250000    4.000000    3.00000    7.000000    1.000000    4.000000   
max    242.000000    4.000000    3.00000   10.000000    1.000000    5.000000   

           PIWORK      PPISOC      PICHOR         LBS     ...             EMP  \
count  200.000000  199.000000  199.000000  200.000000     ...      200.000000   
mean     3.430000    3.160804    3.407035    0.200000     ...        2.680000   
std      1.091355    1.236757    1.1

In [36]:
# dislay
df.ix[0:10, 0:15]

ID  DUR  FREQ  NRS  RAD  PIDAY  PIWORK  PPISOC  PICHOR  LBS  LBST  FUS  \
0    5    1     2  6.0  0.0      3       4     4.0     3.0    0   NaN  NaN   
1    8    4     2  3.0  1.0      2       2     3.0     3.0    0   NaN  NaN   
2   10    3     3  2.0  1.0      2       2     1.0     2.0    0   NaN  NaN   
3   11    4     2  7.0  1.0      4       4     4.0     4.0    0   NaN  NaN   
4   12    3     3  6.0  1.0      3       4     2.0     3.0    0   NaN  NaN   
5   13    3     3  7.0  1.0      4       4     4.0     4.0    0   NaN  NaN   
6   14    3     1  2.0  0.0      1       2     1.0     2.0    0   NaN  NaN   
7   15    4     2  1.0  0.0      2       1     1.0     2.0    0   NaN  NaN   
8   16    3     2  5.0  0.0      3       3     3.0     3.0    0   NaN  NaN   
9   17    4     2  4.0  0.0      4       4     3.0     4.0    0   NaN  NaN   
10  18    4     2  5.0  1.0      5       5     5.0     5.0    2   3.0  0.0   

    OPI  INJ  EXE  
0   1.0  1.0  1.0  
1   0.0  0.0  1.0  
2   0.0  1.0  1.0  
3   0.0  0.0  1.0  
4   0.0  0.0  0.0  
5   0.0  0.0  1.0  
6   NaN  1.0  1.0  
7   1.0  NaN  1.0  
8   0.0  0.0  NaN  
9   0.0  0.0  1.0  
10  1.0  1.0  1.0

#### Drop columns with a high number of NA values, or have the same answer for most questions

These columns likely correlate to questions with few resonses. See legend above.

In [37]:
df=df.drop(['HIS','NAT','ASA','BL','PAC','W','UNK','NA.','LBST','FUS','BL','PAC','W','UNK','NA.','SMOK'], axis=1)
df=df.drop(['SAHELP'], axis=1)
df.head(10)

ID  DUR  FREQ  NRS  RAD  PIDAY  PIWORK  PPISOC  PICHOR  LBS  ...    SEX  \
0   5    1     2  6.0  0.0      3       4     4.0     3.0    0  ...    0.0   
1   8    4     2  3.0  1.0      2       2     3.0     3.0    0  ...    0.0   
2  10    3     3  2.0  1.0      2       2     1.0     2.0    0  ...    1.0   
3  11    4     2  7.0  1.0      4       4     4.0     4.0    0  ...    1.0   
4  12    3     3  6.0  1.0      3       4     2.0     3.0    0  ...    0.0   
5  13    3     3  7.0  1.0      4       4     4.0     4.0    0  ...    1.0   
6  14    3     1  2.0  0.0      1       2     1.0     2.0    0  ...    1.0   
7  15    4     2  1.0  0.0      2       1     1.0     2.0    0  ...    1.0   
8  16    3     2  5.0  0.0      3       3     3.0     3.0    0  ...    1.0   
9  17    4     2  4.0  0.0      4       4     3.0     4.0    0  ...    0.0   

   EMP  EDU    HT     WT  RACE    PI   FUN   DEP  SLEEP  
0    1    0  72.0  123.0   5.0  14.0  11.0  20.0   11.0  
1    1    6  69.0  175.0   5.0  10.0   7.0  12.0    9.0  
2    1    8  67.0  130.0   5.0   7.0   5.0  10.0   10.0  
3    1    6  62.0  108.0   5.0  16.0  12.0  18.0   17.0  
4    1    4  74.0  260.0   5.0  12.0  12.0   7.0   13.0  
5    1    7  64.0  170.0   5.0  16.0  13.0   6.0   12.0  
6    1    7  66.0  198.0   5.0   6.0   NaN   5.0   13.0  
7    1    3  64.0  125.0   5.0   6.0   NaN   7.0    7.0  
8    1    7  68.0  185.0   5.0  12.0   NaN   NaN    6.0  
9    1    9  68.0  190.0   5.0  15.0   9.0   9.0   10.0  

[10 rows x 48 columns]

#### Set remaining NA values to a default

In [38]:
df.columns

Index(['ID', 'DUR', 'FREQ', 'NRS', 'RAD', 'PIDAY', 'PIWORK', 'PPISOC',
       'PICHOR', 'LBS', 'OPI', 'INJ', 'EXE', 'PSY', 'UNEMP', 'DIS', 'ABD',
       'JOI', 'HEA', 'FIB', 'CHOR', 'STAIR', 'W15', 'ERANDS', 'WORTHL',
       'HELPL', 'DEPRES', 'HOPEL', 'SLEEPQ', 'SREFR', 'SPROB', 'SONSET',
       'CAT.SAFE', 'CAT.NEVER', 'LIT', 'AS', 'ETOH', 'AGE', 'SEX', 'EMP',
       'EDU', 'HT', 'WT', 'RACE', 'PI', 'FUN', 'DEP', 'SLEEP'],
      dtype='object')

In [39]:
# set remaining NA to 0
df.fillna(0, inplace=True)
df.head(10)

ID  DUR  FREQ  NRS  RAD  PIDAY  PIWORK  PPISOC  PICHOR  LBS  ...    SEX  \
0   5    1     2  6.0  0.0      3       4     4.0     3.0    0  ...    0.0   
1   8    4     2  3.0  1.0      2       2     3.0     3.0    0  ...    0.0   
2  10    3     3  2.0  1.0      2       2     1.0     2.0    0  ...    1.0   
3  11    4     2  7.0  1.0      4       4     4.0     4.0    0  ...    1.0   
4  12    3     3  6.0  1.0      3       4     2.0     3.0    0  ...    0.0   
5  13    3     3  7.0  1.0      4       4     4.0     4.0    0  ...    1.0   
6  14    3     1  2.0  0.0      1       2     1.0     2.0    0  ...    1.0   
7  15    4     2  1.0  0.0      2       1     1.0     2.0    0  ...    1.0   
8  16    3     2  5.0  0.0      3       3     3.0     3.0    0  ...    1.0   
9  17    4     2  4.0  0.0      4       4     3.0     4.0    0  ...    0.0   

   EMP  EDU    HT     WT  RACE    PI   FUN   DEP  SLEEP  
0    1    0  72.0  123.0   5.0  14.0  11.0  20.0   11.0  
1    1    6  69.0  175.0   5.0  10.0   7.0  12.0    9.0  
2    1    8  67.0  130.0   5.0   7.0   5.0  10.0   10.0  
3    1    6  62.0  108.0   5.0  16.0  12.0  18.0   17.0  
4    1    4  74.0  260.0   5.0  12.0  12.0   7.0   13.0  
5    1    7  64.0  170.0   5.0  16.0  13.0   6.0   12.0  
6    1    7  66.0  198.0   5.0   6.0   0.0   5.0   13.0  
7    1    3  64.0  125.0   5.0   6.0   0.0   7.0    7.0  
8    1    7  68.0  185.0   5.0  12.0   0.0   0.0    6.0  
9    1    9  68.0  190.0   5.0  15.0   9.0   9.0   10.0  

[10 rows x 48 columns]

#### Define data types

Ensure that survey attribures are interpreted by the correct datat type.

In [40]:
df['ID']=df['ID']
df['DUR']=df['DUR'].astype('int')
df['FREQ']=df['FREQ'].astype('int')
df['NRS']=df['NRS'].astype('int')
df['RAD']=df['RAD'].astype('category')

df['PIDAY']=df['PIDAY'].astype('int')
df['PIWORK']=df['PIWORK'].astype('int')
df['PPISOC']=df['PPISOC'].astype('int')
df['PICHOR']=df['PICHOR'].astype('int')

df['LBS']=df['LBS'].astype('category')
df['`']=df['OPI'].astype('category')
df['INJ']=df['INJ'].astype('category')
df['EXE']=df['EXE'].astype('category')
df['PSY']=df['PSY'].astype('category')
df['UNEMP']=df['UNEMP'].astype('category')
df['DIS']=df['DIS'].astype('category')
df['ABD']=df['ABD'].astype('int')
df['JOI']=df['JOI'].astype('int')
df['HEA']=df['HEA'].astype('int')
df['FIB']=df['FIB'].astype('int')
df['CHOR']=df['CHOR'].astype('int')
df['STAIR']=df['STAIR'].astype('int')
df['W15']=df['W15'].astype('int')
df['ERANDS']=df['ERANDS'].astype('int')
df['WORTHL']=df['WORTHL'].astype('int')
df['HELPL']=df['HELPL'].astype('int')
df['DEPRES']=df['DEPRES'].astype('int')
df['HOPEL']=df['HOPEL'].astype('int')
df['SLEEPQ']=df['SLEEPQ'].astype('int')
df['SREFR']=df['SREFR'].astype('int')
df['SPROB']=df['SPROB'].astype('int')
df['SONSET']=df['SONSET'].astype('int')
df['CAT.SAFE']=df['CAT.SAFE'].astype('int')
df['CAT.NEVER']=df['CAT.NEVER'].astype('int')
df['LIT']=df['LIT'].astype('int')
df['AS']=df['AS'].astype('int')
df['ETOH']=df['ETOH'].astype('int')
df['AGE']=df['AGE'].astype('int')
df['SEX']=df['SEX'].astype('category')
df['EMP']=df['EMP'].astype('category')
df['EDU']=df['EDU'].astype('int')
df['HT']=df['HT'].astype('int')
df['WT']=df['WT'].astype('int')
df['RACE']=df['RACE'].astype('category')
df['PI']=df['PI'].astype('int')
df['FUN']=df['FUN'].astype('int')
df['DEP']=df['DEP'].astype('int')
df['SLEEP']=df['SLEEP'].astype('int')

In [41]:
df.describe()

ID         DUR       FREQ         NRS       PIDAY      PIWORK  \
count  200.000000  200.000000  200.00000  200.000000  200.000000  200.000000   
mean   115.520000    3.530000    2.45000    5.465000    3.330000    3.430000   
std     68.958436    0.686627    0.72118    2.147518    1.061477    1.091355   
min      5.000000    1.000000    1.00000    0.000000    1.000000    1.000000   
25%     57.750000    3.000000    2.00000    4.000000    3.000000    3.000000   
50%    107.500000    4.000000    3.00000    6.000000    3.000000    3.000000   
75%    172.250000    4.000000    3.00000    7.000000    4.000000    4.000000   
max    242.000000    4.000000    3.00000   10.000000    5.000000    5.000000   

           PPISOC      PICHOR         OPI         ABD     ...              AS  \
count  200.000000  200.000000  200.000000  200.000000     ...      200.000000   
mean     3.145000    3.390000    0.580000    0.490000     ...        0.460000   
std      1.253728    1.176786    0.494797    0.633963     ...        0.781957   
min      0.000000    0.000000    0.000000    0.000000     ...        0.000000   
25%      2.000000    2.000000    0.000000    0.000000     ...        0.000000   
50%      3.000000    4.000000    1.000000    0.000000     ...        0.000000   
75%      4.000000    4.000000    1.000000    1.000000     ...        1.000000   
max      5.000000    5.000000    1.000000    2.000000     ...        3.000000   

             ETOH         AGE         EDU          HT          WT          PI  \
count  200.000000  200.000000  200.000000  200.000000  200.000000  200.000000   
mean     0.485000   44.950000    4.905000   62.505000  164.430000   13.175000   
std      0.890923   14.929567    2.106695   17.062109   58.187231    4.353059   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.000000   35.000000    3.000000   63.750000  140.000000   10.750000   
50%      0.000000   46.000000    6.000000   66.000000  165.000000   13.000000   
75%      1.000000   56.250000    6.000000   70.000000  197.250000   16.000000   
max      3.000000   81.000000    9.000000   77.000000  350.000000   20.000000   

              FUN         DEP       SLEEP  
count  200.000000  200.000000  200.000000  
mean     8.600000    3.440000   12.650000  
std      4.484479    4.803307    4.109983  
min      0.000000    0.000000    0.000000  
25%      6.000000    0.000000   10.000000  
50%      9.000000    0.000000   12.000000  
75%     12.000000    5.000000   16.000000  
max     17.000000   20.000000   20.000000  

[8 rows x 38 columns]

### Select relevant subset of features 

Select a subset of features that have been found to have relatively high predictive value based on using recurisve feature elimination that also captures physical function, pain interference, and pain intensity, along with opiod use radiating pain and demographics.

In [42]:
#df[['FREQ', 'RAD', 'INJ', 'EXE', 'DIS', 'JOI', 'CHOR', 'ERANDS', 'LIT', 'SEX', 'OPI']].head()

#data = df[['DUR', 'FREQ', 'NRS', 'RAD', 'JOI', 'LBS', 'PICHOR', 'OPI', 'INJ', 'EXE', 'DIS', 'DEPRES', 'AGE', 'SEX']]
# skip age
data = df[['DUR', 'FREQ', 'NRS', 'RAD', 'JOI', 'LBS', 'PICHOR', 'OPI', 'INJ', 'EXE', 'DIS', 'DEPRES', 'SEX']]
data.head()

DUR  FREQ  NRS  RAD  JOI LBS  PICHOR  OPI  INJ  EXE  DIS  DEPRES  SEX
0    1     2    6  0.0    2   0       3  1.0  1.0  1.0  1.0       5  0.0
1    4     2    3  1.0    1   0       3  0.0  0.0  1.0  0.0       4  0.0
2    3     3    2  1.0    0   0       2  0.0  1.0  1.0  0.0       4  1.0
3    4     2    7  1.0    2   0       4  0.0  0.0  1.0  0.0       5  1.0
4    3     3    6  1.0    2   0       3  0.0  0.0  0.0  0.0       2  0.0


# Learning Bayesian Networks


Previous notebooks showed how Bayesian networks economically encode a probability distribution over a set of variables, and how they can be used e.g. to predict variable states, or to generate new samples from the joint distribution. This section will be about obtaining a Bayesian network, given a set of sample data. Learning a Bayesian network can be split into two problems:

 **Parameter learning:** Given a set of data samples and a DAG that captures the dependencies between the variables, estimate the (conditional) probability distributions of the individual variables.
 
 **Structure learning:** Given a set of data samples, estimate a DAG that captures the dependencies between the variables.
 
This notebook aims to illustrate how parameter learning and structure learning can be done with pgmpy.
Currently, the library supports:
 - Parameter learning for *discrete* nodes:
   - Maximum Likelihood Estimation
   - Bayesian Estimation
 - Structure learning for *discrete*, *fully observed* networks:
   - Score-based structure estimation (BIC/BDeu/K2 score; exhaustive search, hill climb/tabu search)
   - Constraint-based structure estimation (PC)
   - Hybrid structure estimation (MMHC)


## Parameter Learning

Suppose we have the following data:

We will assume the following Bayesian network is provided. Naive Bayes network assuming all variables are conditioanlly indepedent givin OPI.

In [43]:
from pgmpy.models import BayesianModel

network = [('OPI','DUR'), ('OPI','FREQ'), ('OPI','NRS'), ('OPI','RAD'), ('OPI','JOI'), ('OPI','LBS'), ('OPI','PICHOR'), ('OPI','INJ'), ('OPI','EXE'), ('OPI','DIS'), ('OPI','DEPRES'), ('OPI','SEX')]   

model = BayesianModel(network)  

Parameter learning is the task to estimate the values of the conditional probability distributions (CPDs), for the variables `fruit`, `size`, and `tasty`. 

#### State counts
To make sense of the given data, we can start by counting how often each state of the variable occurs. If the variable is dependent on parents, the counts are done conditionally on the parents states, i.e. for seperately for each parent configuration:

In [44]:
from pgmpy.estimators import ParameterEstimator
pe = ParameterEstimator(model, data)
print("\n", pe.state_counts('OPI'))  # unconditional
print("\n", pe.state_counts('FREQ'))  # conditional on OPI
print("\n", pe.state_counts('NRS'))  # conditional on OPI
print("\n", pe.state_counts('RAD'))  # conditional on OPI
print("\n", pe.state_counts('JOI'))  # conditional on OPI
print("\n", pe.state_counts('LBS'))  # conditional on OPI
print("\n", pe.state_counts('PICHOR'))  # conditional on OPI
print("\n", pe.state_counts('INJ'))  # conditional on OPI
print("\n", pe.state_counts('EXE'))  # conditional on OPI
print("\n", pe.state_counts('DIS'))  # conditional on OPI
print("\n", pe.state_counts('DEPRES'))  # conditional on OPI
print("\n", pe.state_counts('SEX'))  # conditional on OPI


      OPI
0.0   84
1.0  116

 OPI   0.0  1.0
FREQ          
1      15   12
2      22   34
3      47   70

 OPI   0.0   1.0
NRS            
0     1.0   0.0
1     0.0   3.0
2     8.0   6.0
3    12.0  13.0
4    14.0  13.0
5    11.0  15.0
6    11.0  21.0
7    19.0  20.0
8     6.0  12.0
9     1.0   9.0
10    1.0   4.0

 OPI  0.0  1.0
RAD          
0.0   57   59
1.0   27   57

 OPI  0.0  1.0
JOI          
0     22   21
1     39   45
2     23   50

 OPI  0.0  1.0
LBS          
0     80   92
1      1   15
2      3    9

 OPI      0.0   1.0
PICHOR            
0        0.0   1.0
1        6.0   4.0
2       19.0  21.0
3       24.0  22.0
4       26.0  39.0
5        9.0  29.0

 OPI  0.0  1.0
INJ          
0.0   71   56
1.0   13   60

 OPI  0.0  1.0
EXE          
0.0   27   10
1.0   57  106

 OPI  0.0  1.0
DIS          
0.0   82   88
1.0    2   28

 OPI     0.0  1.0
DEPRES          
0        25   39
1        19   22
2        27   23
4        10   22
5         3   10

 OPI  0.0  1.0
SEX          
0.0

We can see, for example, that as many apples as bananas were observed and that `5` large bananas were tasty, while only `1` was not.

#### Maximum Likelihood Estimation

A natural estimate for the CPDs is to simply use the *relative frequencies*, with which the variable states have occured. 

This approach is *Maximum Likelihood Estimation (MLE)*. According to MLE, we should fill the CPDs in such a way, that $P(\text{data}|\text{model})$ is maximal. This is achieved when using the *relative frequencies*. See [1], section 17.1 for an introduction to ML parameter estimation. pgmpy supports MLE as follows:

In [45]:
from pgmpy.estimators import MaximumLikelihoodEstimator
mle = MaximumLikelihoodEstimator(model, data)
print("\n", mle.estimate_cpd('OPI'))  # unconditional
print("\n", mle.estimate_cpd('FREQ'))  # conditional on OPI
print("\n", mle.estimate_cpd('NRS'))  # conditional on OPI
print("\n", mle.estimate_cpd('RAD'))  # conditional on OPI
print("\n", mle.estimate_cpd('JOI'))  # conditional on OPI
print("\n", mle.estimate_cpd('LBS'))  # conditional on OPI
print("\n", mle.estimate_cpd('PICHOR'))  # conditional on OPI
print("\n", mle.estimate_cpd('INJ'))  # conditional on OPI
print("\n", mle.estimate_cpd('EXE'))  # conditional on OPI
print("\n", mle.estimate_cpd('DIS'))  # conditional on OPI
print("\n", mle.estimate_cpd('DEPRES'))  # conditional on OPI
print("\n", mle.estimate_cpd('SEX'))  # conditional on OPI


 ╒══════════╤══════╕
│ OPI(0.0) │ 0.42 │
├──────────┼──────┤
│ OPI(1.0) │ 0.58 │
╘══════════╧══════╛

 ╒═════════╤═════════════════════╤═════════════════════╕
│ OPI     │ OPI(0.0)            │ OPI(1.0)            │
├─────────┼─────────────────────┼─────────────────────┤
│ FREQ(1) │ 0.17857142857142858 │ 0.10344827586206896 │
├─────────┼─────────────────────┼─────────────────────┤
│ FREQ(2) │ 0.2619047619047619  │ 0.29310344827586204 │
├─────────┼─────────────────────┼─────────────────────┤
│ FREQ(3) │ 0.5595238095238095  │ 0.603448275862069   │
╘═════════╧═════════════════════╧═════════════════════╛

 ╒═════════╤══════════════════════╤══════════════════════╕
│ OPI     │ OPI(0.0)             │ OPI(1.0)             │
├─────────┼──────────────────────┼──────────────────────┤
│ NRS(0)  │ 0.011904761904761904 │ 0.0                  │
├─────────┼──────────────────────┼──────────────────────┤
│ NRS(1)  │ 0.0                  │ 0.02586206896551724  │
├─────────┼──────────────────────┼────────

`mle.estimate_cpd(variable)` computes the state counts and divides each cell by the (conditional) sample size. The `mle.get_parameters()`-method returns a list of CPDs for all variable of the model.

The built-in `fit()`-method of `BayesianModel` provides more convenient access to parameter estimators:


In [46]:
# Calibrate all CPDs of `model` using MLE:
model.fit(data, estimator=MaximumLikelihoodEstimator)


While very straightforward, the ML estimator has the problem of *overfitting* to the data. We may not have enough observations to rely on the observed frequencies. If the observed data is not representative for the underlying distribution, ML estimations will be extremly far off. 

When estimating parameters for Bayesian networks, lack of data is a frequent problem. Even if the total sample size is very large, the fact that state counts are done conditionally for each parents configuration causes immense fragmentation. If a variable has 3 parents that can each take 10 states, then state counts will be done seperately for `10^3 = 1000` parents configurations. This makes MLE very fragile and unstable for learning Bayesian Network parameters. A way to mitigate MLE's overfitting is *Bayesian Parameter Estimation*.

#### Bayesian Parameter Estimation

The Bayesian Parameter Estimator starts with already existing prior CPDs, that express our beliefs about the variables *before* the data was observed. Those "priors" are then updated, using the state counts from the observed data. See [1], Section 17.3 for a general introduction to Bayesian estimators.

One can think of the priors as consisting in *pseudo state counts*, that are added to the actual counts before normalization.
Unless one wants to encode specific beliefs about the distributions of the variables, one commonly chooses uniform priors, i.e. ones that deem all states equiprobable.

A very simple prior is the so-called *K2* prior, which simply adds `1` to the count of every single state.
A somewhat more sensible choice of prior is *BDeu* (Bayesian Dirichlet equivalent uniform prior). For BDeu we need to specify an *equivalent sample size* `N` and then the pseudo-counts are the equivalent of having observed `N` uniform samples of each variable (and each parent configuration). In pgmpy:


In [47]:
from pgmpy.estimators import BayesianEstimator
est = BayesianEstimator(model, data)

equivalent_sample_size = 20
print("\n", est.estimate_cpd('OPI', prior_type='BDeu', equivalent_sample_size=equivalent_sample_size))  # unconditional
print("\n", est.estimate_cpd('FREQ', prior_type='BDeu', equivalent_sample_size=equivalent_sample_size))  # conditional on OPI
print("\n", est.estimate_cpd('NRS', prior_type='BDeu', equivalent_sample_size=equivalent_sample_size))  # conditional on OPI
print("\n", est.estimate_cpd('RAD', prior_type='BDeu', equivalent_sample_size=equivalent_sample_size))  # conditional on OPI
print("\n", est.estimate_cpd('JOI', prior_type='BDeu', equivalent_sample_size=equivalent_sample_size))  # conditional on OPI
print("\n", est.estimate_cpd('LBS', prior_type='BDeu', equivalent_sample_size=equivalent_sample_size))  # conditional on OPI
print("\n", est.estimate_cpd('PICHOR', prior_type='BDeu', equivalent_sample_size=equivalent_sample_size))  # conditional on OPI
print("\n", est.estimate_cpd('INJ', prior_type='BDeu', equivalent_sample_size=equivalent_sample_size))  # conditional on OPI
print("\n", est.estimate_cpd('EXE', prior_type='BDeu', equivalent_sample_size=equivalent_sample_size))  # conditional on OPI
print("\n", est.estimate_cpd('DIS', prior_type='BDeu', equivalent_sample_size=equivalent_sample_size))  # conditional on OPI
print("\n", est.estimate_cpd('DEPRES', prior_type='BDeu', equivalent_sample_size=equivalent_sample_size))  # conditional on OPI
print("\n", est.estimate_cpd('SEX', prior_type='BDeu', equivalent_sample_size=equivalent_sample_size))  # conditional on OPI



 ╒══════════╤══════════╕
│ OPI(0.0) │ 0.427273 │
├──────────┼──────────┤
│ OPI(1.0) │ 0.572727 │
╘══════════╧══════════╛

 ╒═════════╤═════════════════════╤═════════════════════╕
│ OPI     │ OPI(0.0)            │ OPI(1.0)            │
├─────────┼─────────────────────┼─────────────────────┤
│ FREQ(1) │ 0.19503546099290778 │ 0.1216931216931217  │
├─────────┼─────────────────────┼─────────────────────┤
│ FREQ(2) │ 0.2695035460992908  │ 0.29629629629629634 │
├─────────┼─────────────────────┼─────────────────────┤
│ FREQ(3) │ 0.5354609929078015  │ 0.582010582010582   │
╘═════════╧═════════════════════╧═════════════════════╛

 ╒═════════╤══════════════════════╤══════════════════════╕
│ OPI     │ OPI(0.0)             │ OPI(1.0)             │
├─────────┼──────────────────────┼──────────────────────┤
│ NRS(0)  │ 0.020309477756286273 │ 0.007215007215007216 │
├─────────┼──────────────────────┼──────────────────────┤
│ NRS(1)  │ 0.009671179883945842 │ 0.031024531024531028 │
├─────────┼───────────

The estimated values in the CPDs are now more conservative. In particular, the estimate for  OPI 0/1 was 0.42/0.58 with a sample size = 20 is smoothed to 0.43/0.57. Large values for equivalent sample size or reducing the N samples would result in increasing more conservative estiamtes.

`BayesianEstimator`, too, can be used via the `fit()`-method. Full example:

In [48]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.estimators import BayesianEstimator

model.fit(data, estimator=BayesianEstimator, prior_type="BDeu") 
# default equivalent_sample_size=5
for cpd in model.get_cpds():
    print(cpd)


╒═══════════╤═════════════════════╤═════════════════════╕
│ OPI       │ OPI(0.0)            │ OPI(1.0)            │
├───────────┼─────────────────────┼─────────────────────┤
│ DEPRES(0) │ 0.2947976878612717  │ 0.3333333333333333  │
├───────────┼─────────────────────┼─────────────────────┤
│ DEPRES(1) │ 0.2254335260115607  │ 0.189873417721519   │
├───────────┼─────────────────────┼─────────────────────┤
│ DEPRES(2) │ 0.3179190751445087  │ 0.19831223628691982 │
├───────────┼─────────────────────┼─────────────────────┤
│ DEPRES(4) │ 0.12138728323699421 │ 0.189873417721519   │
├───────────┼─────────────────────┼─────────────────────┤
│ DEPRES(5) │ 0.04046242774566474 │ 0.08860759493670886 │
╘═══════════╧═════════════════════╧═════════════════════╛
╒══════════╤═════════════════════╤════════════════════╕
│ OPI      │ OPI(0.0)            │ OPI(1.0)           │
├──────────┼─────────────────────┼────────────────────┤
│ DIS(0.0) │ 0.9624277456647399  │ 0.7531645569620253 │
├──────────┼──────────


## Structure Learning

To learn model structure (a DAG) from a data set, there are two broad techniques:

 - score-based structure learning
 - constraint-based structure learning

The combination of both techniques allows further improvement:
 - hybrid structure learning

We briefly discuss all approaches and give examples.

### Score-based Structure Learning


This approach construes model selection as an optimization task. It has two building blocks:

- A _scoring function_ $s_D\colon M \to \mathbb R$ that maps models to a numerical score, based on how well they fit to a given data set $D$.
- A _search strategy_ to traverse the search space of possible models $M$ and select a model with optimal score.


#### Scoring functions

Commonly used scores to measure the fit between model and data are _Bayesian Dirichlet scores_ such as *BDeu* or *K2* and the _Bayesian Information Criterion_ (BIC, also called MDL). See [1], Section 18.3 for a detailed introduction on scores. As before, BDeu is dependent on an equivalent sample size.

In [49]:
import pandas as pd
import numpy as np
from pgmpy.estimators import BdeuScore, K2Score, BicScore
from pgmpy.models import BayesianModel

# create random data sample with 3 variables, where Z is dependent on X, Y:
data0 = pd.DataFrame(np.random.randint(0, 4, size=(5000, 2)), columns=list('XY'))
data0['Z'] = data0['X'] + data0['Y']

bdeu = BdeuScore(data0, equivalent_sample_size=5)
k2 = K2Score(data0)
bic = BicScore(data0)

model1 = BayesianModel([('X', 'Z'), ('Y', 'Z')])  # X -> Z <- Y
model2 = BayesianModel([('X', 'Z'), ('X', 'Y')])  # Y <- X -> Z


print(bdeu.score(model1))
print(k2.score(model1))
print(bic.score(model1))

print(bdeu.score(model2))
print(k2.score(model2))
print(bic.score(model2))


-13934.716633404805
-14325.39375557494
-14290.759744
-20893.440702569555
-20920.27712071741
-20937.4995007


While the scores vary slightly, we can see that the correct `model1` has a much higher score than `model2`.
Importantly, these scores _decompose_, i.e. they can be computed locally for each of the variables given their potential parents, independent of other parts of the network:

In [50]:
print(bdeu.local_score('Z', parents=[]))
print(bdeu.local_score('Z', parents=['X']))
print(bdeu.local_score('Z', parents=['X', 'Y']))

-9217.130247825968
-6989.664089188232
-57.11390342056143


#### Search strategies
The search space of DAGs is super-exponential in the number of variables and the above scoring functions allow for local maxima. The first property makes exhaustive search intractable for all but very small networks, the second prohibits efficient local optimization algorithms to always find the optimal structure. Thus, identifiying the ideal structure is often not tractable. Despite these bad news, heuristic search strategies often yields good results.

If only few nodes are involved (read: less than 5), `ExhaustiveSearch` can be used to compute the score for every DAG and returns the best-scoring one:

In [51]:
from pgmpy.estimators import ExhaustiveSearch

# reduce number of random variables
from pgmpy.estimators import ExhaustiveSearch

#data2 = data[['OPI', 'INJ', 'LBS','PICHOR', 'NRS', 'RAD', 'JOI']]

data2 = data[['INJ', 'LBS','OPI']]

bdeu = BdeuScore(data2, equivalent_sample_size=5)
k2 = K2Score(data2)
bic = BicScore(data2)

es = ExhaustiveSearch(data2, scoring_method=bic)
best_model = es.estimate()
print(best_model.edges())

print("\nAll DAGs by score:")
count = 0
for score, dag in reversed(es.all_scores()):
    print(score, dag.edges())
    count += 1
    if count>10:
        break;

[('INJ', 'LBS'), ('INJ', 'OPI')]

All DAGs by score:
-349.556280976 [('LBS', 'INJ'), ('INJ', 'OPI')]
-349.556280976 [('OPI', 'INJ'), ('INJ', 'LBS')]
-349.556280976 [('INJ', 'LBS'), ('INJ', 'OPI')]
-356.948794298 [('OPI', 'LBS'), ('OPI', 'INJ'), ('INJ', 'LBS')]
-356.948794298 [('LBS', 'INJ'), ('OPI', 'LBS'), ('OPI', 'INJ')]
-356.948794298 [('LBS', 'OPI'), ('LBS', 'INJ'), ('OPI', 'INJ')]
-356.948794298 [('LBS', 'OPI'), ('LBS', 'INJ'), ('INJ', 'OPI')]
-356.948794298 [('OPI', 'LBS'), ('INJ', 'LBS'), ('INJ', 'OPI')]
-356.948794298 [('LBS', 'OPI'), ('INJ', 'LBS'), ('INJ', 'OPI')]
-358.417818546 [('LBS', 'INJ'), ('OPI', 'INJ')]
-360.156180231 [('LBS', 'INJ'), ('OPI', 'LBS')]


Once more nodes are involved, one needs to switch to heuristic search. `HillClimbSearch` implements a greedy local search that starts from the DAG `start` (default: disconnected DAG) and proceeds by iteratively performing single-edge manipulations that maximally increase the score. The search terminates once a local maximum is found.




In [52]:
from pgmpy.estimators import HillClimbSearch

hc = HillClimbSearch(data, scoring_method=BicScore(data))
best_model = hc.estimate()
print(best_model.edges())

[('PICHOR', 'RAD'), ('INJ', 'LBS'), ('INJ', 'DIS'), ('INJ', 'FREQ'), ('OPI', 'DIS'), ('OPI', 'INJ'), ('OPI', 'EXE'), ('RAD', 'JOI'), ('RAD', 'OPI')]


To enforce a wider exploration of the search space, the search can be enhanced with a tabu list. The list keeps track of the last `n` modfications; those are then not allowed to be reversed, regardless of the score. Additionally a `white_list` or `black_list` can be supplied to restrict the search to a particular subset or to exclude certain edges. The parameter `max_indegree` allows to restrict the maximum number of parents for each node.


### Constraint-based Structure Learning

A different, but quite straightforward approach to build a DAG from data is this:

1. Identify independencies in the data set using hypothesis tests 
2. Construct DAG (pattern) according to identified independencies

#### (Conditional) Independence Tests

Independencies in the data can be identified using chi2 conditional independence tests. To this end, constraint-based estimators in pgmpy have a `test_conditional_independence(X, Y, Zs)`-method, that performs a hypothesis test on the data sample. It allows to check if `X` is independent from `Y` given a set of variables `Zs`:

In [21]:
from pgmpy.estimators import ConstraintBasedEstimator

# sample data
data00 = pd.DataFrame(np.random.randint(0, 3, size=(2500, 8)), columns=list('ABCDEFGH'))
data00['A'] += data00['B'] + data00['C']
data00['H'] = data00['G'] - data00['A']
data00['E'] *= data00['F']

est = ConstraintBasedEstimator(data00)

print(est.test_conditional_independence('B', 'H'))          # dependent
print(est.test_conditional_independence('B', 'E'))          # independent
print(est.test_conditional_independence('B', 'H', ['A']))   # independent
print(est.test_conditional_independence('A', 'G'))          # independent
print(est.test_conditional_independence('A', 'G',  ['H']))  # dependent

(662.12032349956701, 6.2651022777027284e-123, True)
(4.6402848339816627, 0.94732925952156943, True)
(15.872232810066384, 0.99997026665122879, True)
(5.7183082660601627, 0.99922794955246963, True)
(4618.0, 0.0, True)


In [22]:
from pgmpy.estimators import ConstraintBasedEstimator

est = ConstraintBasedEstimator(data)

# ['DUR', 'FREQ', 'NRS', 'RAD', 'JOI', 'LBS', 'PICHOR', 'OPI', 'INJ', 'EXE', 'DIS', 'DEPRES', 'SEX']

print(est.test_conditional_independence('OPI', 'INJ'))         
print(est.test_conditional_independence('OPI', 'LBS'))          
print(est.test_conditional_independence('LBS', 'INJ'))  

(27.618973230613527, 4.3662414579369054e-06, True)
(11.255346164127243, 0.046545787789089264, True)
(40.180558029163208, 1.3732531235567979e-07, True)


`test_conditional_independence()` returns a tripel `(chi2, p_value, sufficient_data)`, consisting in the computed chi2 test statistic, the `p_value` of the test, and a heuristig flag that indicates if the sample size was sufficient. The `p_value` is the probability of observing the computed chi2 statistic (or an even higher chi2 value), given the null hypothesis that X and Y are independent given Zs.

This can be used to make independence judgements, at a given level of significance:

In [23]:
def is_independent(X, Y, Zs=[], significance_level=0.05):
    return est.test_conditional_independence(X, Y, Zs)[1] >= significance_level

print(is_independent('OPI', 'INJ'))
print(is_independent('OPI', 'LBS'))
print(is_independent('LBS', 'INJ'))


False
False
False


#### DAG (pattern) construction

With a method for independence testing at hand, we can construct a DAG from the data set in three steps:
1. Construct an undirected skeleton - `estimate_skeleton()`
2. Orient compelled edges to obtain partially directed acyclid graph (PDAG; I-equivalence class of DAGs) - `skeleton_to_pdag()`
3. Extend DAG pattern to a DAG by conservatively orienting the remaining edges in some way - `pdag_to_dag()`

Step 1.&2. form the so-called PC algorithm, see [2], page 550. PDAGs are `DirectedGraph`s, that may contain both-way edges, to indicate that the orientation for the edge is not determined.




In [24]:
skel, seperating_sets = est.estimate_skeleton(significance_level=0.01)
print("Undirected edges: ", skel.edges())

pdag = est.skeleton_to_pdag(skel, seperating_sets)
print("PDAG edges:       ", pdag.edges())

model = est.pdag_to_dag(pdag)
print("DAG edges:        ", model.edges())

Undirected edges:  [('PICHOR', 'NRS'), ('INJ', 'OPI'), ('INJ', 'LBS'), ('JOI', 'RAD')]
PDAG edges:        [('PICHOR', 'NRS'), ('OPI', 'INJ'), ('JOI', 'RAD'), ('RAD', 'JOI'), ('LBS', 'INJ'), ('NRS', 'PICHOR')]
DAG edges:         [('OPI', 'INJ'), ('RAD', 'JOI'), ('LBS', 'INJ'), ('NRS', 'PICHOR')]


The `estimate()`-method provides a shorthand for the three steps above and directly returns a `BayesianModel`:



In [25]:
print(est.estimate(significance_level=0.01).edges())

[('OPI', 'INJ'), ('RAD', 'JOI'), ('LBS', 'INJ'), ('NRS', 'PICHOR')]


The `estimate_from_independencies()`-method can be used to construct a `BayesianModel` from a provided *set of independencies* (see class documentation for further features & methods):

In [26]:
from pgmpy.independencies import Independencies

ind = Independencies([('LBS', 'INJ'), ('JOI', 'RAD'), ('OPI', 'INJ'), ('PICHOR', 'NRS')])
ind = ind.closure()  # required (!) for faithfulness

model = ConstraintBasedEstimator.estimate_from_independencies(['LBS', 'INJ', 'JOI', 'RAD', 'OPI', 'INJ', 'PICHOR', 'NRS'], ind)

print(model.edges())

[('LBS', 'INJ'), ('LBS', 'PICHOR'), ('LBS', 'NRS'), ('OPI', 'LBS'), ('OPI', 'JOI'), ('OPI', 'INJ'), ('OPI', 'PICHOR'), ('OPI', 'NRS'), ('PICHOR', 'NRS'), ('RAD', 'JOI'), ('RAD', 'OPI'), ('RAD', 'PICHOR'), ('RAD', 'NRS'), ('JOI', 'PICHOR'), ('JOI', 'NRS'), ('INJ', 'PICHOR'), ('INJ', 'NRS')]


### Evaluate models

In [33]:
bdeu = BdeuScore(data, equivalent_sample_size=5)
k2 = K2Score(data)
bic = BicScore(data)

# naive bayes model
model0 = BayesianModel([('OPI','DUR'), ('OPI','FREQ'), ('OPI','NRS'), ('OPI','RAD'), ('OPI','JOI'), ('OPI','LBS'), ('OPI','PICHOR'), ('OPI','INJ'), ('OPI','EXE'), ('OPI','DIS'), ('OPI','DEPRES'), ('OPI','SEX')])
# hill climbing model
model1 = BayesianModel([('RAD', 'PICHOR'), ('RAD', 'JOI'), ('RAD', 'OPI'), ('OPI', 'DIS'), ('OPI', 'INJ'), ('OPI', 'EXE'), ('INJ', 'DIS'), ('INJ', 'LBS'), ('INJ', 'FREQ')])
# constraint model learned from pair-wise independencies
model2 = BayesianModel([('LBS', 'INJ'), ('JOI', 'RAD'), ('OPI', 'INJ'), ('PICHOR', 'NRS')])  
# model built using indepencies estimated from model 2
model3 = BayesianModel([('PICHOR', 'OPI'), ('PICHOR', 'NRS'), ('INJ', 'PICHOR'), ('INJ', 'OPI'), ('INJ', 'NRS'), ('RAD', 'PICHOR'), ('RAD', 'JOI'), ('RAD', 'OPI'), ('RAD', 'NRS'), ('LBS', 'PICHOR'), ('LBS', 'INJ'), ('LBS', 'OPI'), ('LBS', 'NRS'), ('OPI', 'NRS'), ('JOI', 'PICHOR'), ('JOI', 'OPI'), ('JOI', 'NRS')])  
# domain expert model
model4 = BayesianModel([('DIS', 'OPI'), ('DIS', 'INJ'), ('OPI', 'RAD'), ('OPI', 'INJ'), ('RAD', 'PICHOR'), ('RAD', 'JOI'), ('INJ', 'FREQ'), ('INJ', 'LBS')])

print("model0 Naive Bayes")
print(bdeu.score(model0))
print(k2.score(model0))
print(bic.score(model0))

print("model1 Hill Climbing")
print(bdeu.score(model1))
print(k2.score(model1))
print(bic.score(model1))

print("model2 Constraint Learning")
print(bdeu.score(model2))
print(k2.score(model2))
print(bic.score(model2))

print("model3 Independencies from Constraint Learned Model")
print(bdeu.score(model3))
print(k2.score(model3))
print(bic.score(model3))

print("model4 Medical domain expert defined model")
print(bdeu.score(model4))
print(k2.score(model4))
print(bic.score(model4))

model0 Naive Bayes
-2503.840315758571
-2491.6320735221907
-2541.02633182
model1 Hill Climbing
-1361.3205978300227
-1362.1412456048588
-1377.64616565
model2 Constraint Learning
-1474.6274320755251
-1445.220201410989
-1550.31881365
model3 Independencies from Constraint Learned Model
-1966.341356309149
-1507.7318695562417
-13530.0982644
model4 Independencies from Constraint Learned Model
-1269.666550422272
-1268.2079060428323
-1285.53468756


Model 4, defined by Dr. Meredith Adams provided the best fit to the data, followed by Hill Climbing using Bayesian Information criteria.

PC PDAG construction is only guaranteed to work under the assumption that the identified set of independencies is *faithful*, i.e. there exists a DAG that exactly corresponds to it. Spurious dependencies in the data set can cause the reported independencies to violate faithfulness. It can happen that the estimated PDAG does not have any faithful completions (i.e. edge orientations that do not introduce new v-structures). In that case a warning is issued.


### Hybrid Structure Learning

The MMHC algorithm [3] combines the constraint-based and score-based method. It has two parts:

1. Learn undirected graph skeleton using the constraint-based construction procedure MMPC
2. Orient edges using score-based optimization (BDeu score + modified hill-climbing)

This method is not functional.

`MmhcEstimator.estimate()` is a shorthand for both steps and directly estimates a `BayesianModel`.

### Conclusion

This notebook aimed to give an overview of pgmpy's estimators for learning Bayesian network structure and parameters. For more information about the individual functions see their docstring documentation. If you used pgmpy's structure learning features to satisfactorily learn a non-trivial network from real data, feel free to drop us an eMail via the mailing list or just open a Github issue. We'd like to put your network in the examples-section!

### References

[1] Koller & Friedman, Probabilistic Graphical Models - Principles and Techniques, 2009

[2] Neapolitan, [Learning Bayesian Networks](http://www.cs.technion.ac.il/~dang/books/Learning%20Bayesian%20Networks&#40;Neapolitan,%20Richard&#41;.pdf), 2003

[3] Tsamardinos et al., [The max-min hill-climbing BN structure learning algorithm](http://www.dsl-lab.org/supplements/mmhc_paper/paper_online.pdf), 2005

